In [2]:
%load_ext autoreload
%autoreload 2
from characterHelpers import extractCharacter,isCharacter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    whole_text=""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

        # Get the number of pages in the PDF
        num_pages = len(pdf_reader.pages)

        # Iterate through each page and extract text
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            whole_text+=text
            
            # You can process the extracted text as needed (print, store, etc.)
            # print(f"Page {page_num + 1}:\n{text}\n")

        return whole_text

# Replace 'your_pdf_file.pdf' with the path to your PDF file
pdf_file_path = '../data/AVATAR_JamesCameron.pdf'
whole_text=extract_text_from_pdf(pdf_file_path)


In [6]:
whole_text

"    AVATAR  written by  James Cameron                                 ALL RIGHTS RESERVED. COPYRIGHT ©2007 TWENTIETH CENTURY FOX FILM CORPORATION.  NO PORTION OF THIS SCRIPT MAY BE PERFORMED, PUBLISHED, REPRODUCED, SOLD, OR DISTRIBUTED BY ANY MEANS, OR QUOTED OR PUBLISHED IN ANY MEDIUM, INCLUDING ANY WEB SITE, WITHOUT THE PRIOR WRITTEN CONSENT OF TWENTIETH CENTURY FOX FILM CORPORATION. \u2029THE SOUND OF DRUMS, from a great distance, growing louder.FADE IN:WE ARE FLYING through mist, a dimly glimpsed forest below.VOICE (V.O.)When I was lying there in the VA hospital, with a big hole blown through the middle of my life, I started having these dreams of flying.We are very low over the forest now, gliding fast, the drums BUILDING to a PEAK --VOICE (V.O.)Sooner or later though, you always have to wake up...CUT TO:EXT. CITY - NIGHTA SCREECH OF BRAKES as a vehicle WIPES FRAME, revealing --JAKE SULLY, a scarred and scruffy combat vet, sitting in a beat up carbon-fiber wheelchair.  At 22, his

In [ ]:
import json
import re

HEADING_ENUM = ["EXT./INT.", "EXT./INT.", "INT./EXT.", "EXT/INT", "INT/EXT", "INT.", "EXT.", "INT --", "EXT --"]

def is_heading(line):
    for heading in HEADING_ENUM:
        if not line.endswith(heading) and heading in line:
            return True
    return False

def extract_time_and_scene_data(text):
    time_vocab = "|".join([
        "NIGHT", "AFTERNOON", "MORNING", "DAYS", "DAY", "NIGHT", "DAYS", "DAY", "ANOTHER DAY", "LATER", "NIGHT",
        "SAME", "CONTINUOUS", "MOMENTS LATER", "LATER", "SUNSET",
    ])
    regex = '[-,]?[ ]?(DAWN|DUSK|((LATE|EARLY) )?' + time_vocab + ')|\d{4}'
    find_time = re.search(regex, text)

    txt = list(filter(lambda x: len(x) > 0, [x.strip("-,. ") for x in text[find_time.end():].split()])) if find_time else None
    scene_data = {
        "text": " ".join(txt) if txt else "",
    }
    return txt, scene_data

def extract_heading(text):
    region_match = re.search('((?:.* )?(?:EXT[\\.]?\\/INT[\\.]?|INT[\\.]?\\/EXT[\\.]?|INT(?:\\.| --)|EXT(?:\\.| --)))', text)
    region = region_match.groups()[0] if region_match else ""
    time, scene_data = extract_time_and_scene_data(text)

    location = text.replace(region, "")
    if time and len(time) > 0:
        location = location[:location.index(time[0])]

    if len(region) > 0 and region[0].isdigit():
        region = region.lstrip('0123456789.- ')
        location = location.rstrip('0123456789.- ') if location else location
    time = time[:-1] if time and time[-1].isdigit() else time

    scene_info = {
        "region": region.strip(),
        "location": location.strip("-,. "),
        "text": " ".join(time) if time else "",
    }

    return scene_data, scene_info

def process_script(script_text):
    script_lines = script_text.split('\n')
    result = []
    current_data = {"text": ""}
    scene_counter = 1  # Initialize the scene counter
    for line in script_lines:
        if is_heading(line):
            if current_data["text"]:
                scene_detail, scene_info = extract_heading(current_data["text"])
                result.append({scene_counter: {'scene_info': scene_info, 'scene_detail': scene_detail}})
                current_data = {"text": ""}
                scene_counter += 1  # Increment the scene counter
            current_data["text"] += line
        else:
            current_data["text"] += " " + line

    # Handle the last block of text
    if current_data["text"]:
        scene_detail, scene_info = extract_heading(current_data["text"])
        result.append({scene_counter: {'scene_info': scene_info, 'scene_detail': scene_detail}})

    return result

# Example usage:
script_text = """
EXT. Forest - DAY
Some action in the forest during the day.
EXT. City Street - NIGHT
A different scene in the city at night.
"""

result = process_script(whole_text)

# Save the result to a JSON file
with open("output.json", "w") as json_file:
    json.dump(result, json_file, indent=2)

print("Result saved to output.json")


In [24]:
result

[{1: {'scene_info': {'region': 'EXT.',
    'location': 'AVATAR  written by  James Cameron                                 ALL RIGHTS RESERVED. COPYRIGHT ©2007',
    'text': "TWENTIETH CENTURY FOX FILM CORPORATION NO PORTION OF THIS SCRIPT MAY BE PERFORMED PUBLISHED REPRODUCED SOLD OR DISTRIBUTED BY ANY MEANS OR QUOTED OR PUBLISHED IN ANY MEDIUM INCLUDING ANY WEB SITE WITHOUT THE PRIOR WRITTEN CONSENT OF TWENTIETH CENTURY FOX FILM CORPORATION THE SOUND OF DRUMS from a great distance growing louder.FADE IN:WE ARE FLYING through mist a dimly glimpsed forest below.VOICE (V.O.)When I was lying there in the VA hospital with a big hole blown through the middle of my life I started having these dreams of flying.We are very low over the forest now gliding fast the drums BUILDING to a PEAK VOICE (V.O.)Sooner or later though you always have to wake up...CUT TO:EXT CITY NIGHTA SCREECH OF BRAKES as a vehicle WIPES FRAME revealing JAKE SULLY a scarred and scruffy combat vet sitting in a beat up carb

In [22]:
with open("output.json", "w") as json_file:
    json.dump(result, json_file, indent=2)

print("Result saved to output.json")

Result saved to output.json


In [12]:
for index, data in enumerate(result, start=1):
    scene_text=data[index]['scene_detail']['text']

    if len(scene_text)>0:
        break
    


In [13]:
x_text=result[2][3]['scene_detail']['text']

In [14]:
def getParenthetical(text):
    """splits dialogue string into list, seperating any containing parenthetical(s)"""

    return list(
        filter(lambda x: len(x.strip()) > 0, re.split(r'(\([^)]+\))', text))
    )

In [15]:
isCharacter({"text":"RALEIGH"})

True